In [1]:
from torchvision import datasets, transforms

In [2]:
train_data = datasets.GTSRB(
    root='./data',
    split='train',
    download=True
)

100%|██████████| 187M/187M [00:15<00:00, 11.8MB/s]


Extracting data/gtsrb/GTSRB-Training_fixed.zip to data/gtsrb


In [3]:
test_data = datasets.GTSRB(
    root='./data',
    split='test',
    download=True
)

100%|██████████| 89.0M/89.0M [00:05<00:00, 17.7MB/s]


Extracting data/gtsrb/GTSRB_Final_Test_Images.zip to data/gtsrb


100%|██████████| 99.6k/99.6k [00:00<00:00, 211kB/s]


Extracting data/gtsrb/GTSRB_Final_Test_GT.zip to data/gtsrb


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Assuming train_data and test_data are already defind as in the previous example

In [5]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((32, 32)),  #Resize images to a consistent size
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
])

train_data.transform = transform
test_data.transform = transform

Create data loaders

In [6]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

Define a simple CNN model

In [7]:
class GTSRBNet(nn.Module):
  def __init__(self):
    super(GTSRBNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
    self.relu1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.relu2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1 = nn.Linear(64 * 8 * 8, 128)
    self.relu3 = nn.ReLU()
    self.fc2 = nn.Linear(128, 43) # 43 classes in GTSRB

  def forward(self, x):
    x = self.pool1(self.relu1(self.conv1(x)))
    x = self.pool2(self.relu2(self.conv2(x)))
    x = x.view(-1, 64 * 8 * 8)
    x = self.relu3(self.fc1(x))
    x = self.fc2(x)
    return x

Initilize the model, loss function, and optimizer

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GTSRBNet().to(device)
ceiterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Training loop

In [9]:
num_epochs = 14  # Adjust as needed

for epoch in range(num_epochs):
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = ceiterion(outputs, labels)
    loss.backward()
    optimizer.step()

  print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/14], Loss: 0.5974
Epoch [2/14], Loss: 0.1055
Epoch [3/14], Loss: 0.0114
Epoch [4/14], Loss: 0.0305
Epoch [5/14], Loss: 0.1392
Epoch [6/14], Loss: 0.0053
Epoch [7/14], Loss: 0.0026
Epoch [8/14], Loss: 0.0007
Epoch [9/14], Loss: 0.0437
Epoch [10/14], Loss: 0.0799
Epoch [11/14], Loss: 0.0002
Epoch [12/14], Loss: 0.0004
Epoch [13/14], Loss: 0.0843
Epoch [14/14], Loss: 0.0002


Evaluation

In [10]:
correct = 0
total = 0

with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the test images: {100 * correct/total}%")
print("done!")

Accuracy of the network on the test images: 88.30562153602534%
